In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [11]:
unpickled_df = pd.read_pickle("./engineered_data.pkl")

In [12]:
unpickled_df.columns

Index(['words_length_4', 'words_length_6', 'words_length_8', 'words_length_10',
       'words_length_12', 'mean_word_length', 'variance_word_length',
       'type_token_ratio', 'essay_length', 'num_words', 'num_sentences',
       'mean_sentence_length', 'num_characters', 'fourth_root_num_characters',
       'num_commas', 'num_periods', 'num_exclaim', 'num_question',
       'num_semicolon', 'num_colon', 'vocab_size', 'very_short_sentences',
       'short_sentences', 'medium_sentences', 'long_sentences',
       'variance_sentence_length', 'max_height', 'sum_heights', 'mean_heights',
       'pos_trigram_ratio', 'pos_fourgram_ratio', 'mean_trigram_tfTF',
       'connectives', 'score', 'essay_set', 'essay_id'],
      dtype='object')

In [35]:
# feature_data = .drop('mean_trigram_tfTF', axis=1)
trainingdata = unpickled_df[unpickled_df.essay_set < 7]
trainingdata = trainingdata.dropna()
# classification_data  = classification_data.dropna(subset=['domain1_score'])

In [36]:
X, Y = trainingdata.iloc[:,:-3], trainingdata.loc[:,'score']

In [37]:
x_train,x_test,y_train,y_test = train_test_split(np.asarray(X), np.asarray(Y),test_size=0.2)

In [118]:
# null_columns=classification_data.columns[classification_data.isnull().any()]
# classification_data[null_columns].isnull().sum()

In [38]:
print(sum(np.isnan(y_train)))
print(sum(np.isnan(y_test)))

0
0


In [39]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8547, 33)
(2137, 33)
(8547,)
(2137,)


In [44]:
# gnb = GaussianNB()
lr = LinearRegression(fit_intercept=True)
rf = RandomForestRegressor(n_estimators = 100)

In [45]:
# gnb.fit(x_train, y_train)
lr.fit(x_train, y_train)
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [47]:
scoringFrame_baselines = pd.DataFrame([y_test,rf.predict(x_test), lr.predict(x_test)]).transpose()
scoringFrame_baselines.columns = ['actual','rf','lr']

In [48]:
#Cohen Kappa score as defined by the kaggle challenge/wikipedia
def CohenKappa(actual, predict):
    CohenDF = pd.DataFrame([actual.astype(np.double).round(), np.around(predict.astype(np.double))]).transpose()
    count = len(CohenDF)
    correct = len(CohenDF[CohenDF[actual.name] == CohenDF[predict.name]])
    acc = correct / count
    pe = 0
    for value in CohenDF[actual.name].unique():
        pe += len(CohenDF[CohenDF[actual.name] == value]) * len(CohenDF[CohenDF[predict.name] == value])
    pe = pe / np.square(count)
    return(1 - (1-acc)/(1-pe))

In [52]:
from sklearn.metrics import accuracy_score, mean_squared_error

# print("GNB MSE, Cohen, accuracy: {0}, {1}, {2}".format(mean_squared_error(scoringFrame_baselines.actual, scoringFrame_baselines.gnb),
#                                        CohenKappa(scoringFrame_baselines.actual, scoringFrame_baselines.gnb),
#                                                  accuracy_score(scoringFrame_baselines.actual, scoringFrame_baselines.gnb)))

print("RF MSE, Cohen, accuracy: {0}, {1}".format(mean_squared_error(scoringFrame_baselines.actual, scoringFrame_baselines.rf),
                                       CohenKappa(scoringFrame_baselines.actual, scoringFrame_baselines.rf)))
#                                       accuracy_score(scoringFrame_baselines.actual, scoringFrame_baselines.rf)))

print("LR MSE, Cohen: {0}, {1}".format(mean_squared_error(scoringFrame_baselines.actual, scoringFrame_baselines.lr),
                                       CohenKappa(scoringFrame_baselines.actual, scoringFrame_baselines.lr)))

RF MSE, Cohen, accuracy: 0.025454028317023612, 0.6265134034053828
LR MSE, Cohen: 0.0285115094271961, 0.5653226153369811
